In [1]:
!pip install -q langgraph langchain langchain-google-genai

In [2]:
!pip install -q python-dotenv

In [3]:
!pip install -q langchain-groq

In [4]:
import langchain_groq
print("langchain-groq installed correctly")

langchain-groq installed correctly


In [5]:
from langchain_groq import ChatGroq

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    groq_api_key=GROQ_API_KEY,
    temperature=0
)

In [7]:
from langchain_core.tools import tool
import random

@tool
def flip_coin():
    """Returns either 'Heads' or 'Tails' randomly."""
    result = random.choice(["Heads", "Tails"])
    return f"The coin landed on: {result}"

@tool
def roll_dice():
    """Returns a random number between 1 and 6."""
    result = random.randint(1, 6)
    return f"The die rolled a: {result}"

print("Tools ready!")

Tools ready!
